In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from time import time

In [2]:
path = '/Users/Vishy/Files/AVDatahack/LOM/Data/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
camp = pd.read_csv(path+'campaign_data.csv')
print(train.shape, test.shape, camp.shape)

(1023191, 6) (773858, 4) (52, 9)


In [3]:
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0
3,44_231448,231448,44,05-09-2017 11:36,0,0
4,29_185580,185580,29,01-07-2017 18:01,0,0


In [4]:
test.head()

,id,campaign_id,user_id,send_date
0,63_122715,63,122715,01-02-2018 22:35
1,56_76206,56,76206,02-01-2018 08:15
2,57_96189,57,96189,05-01-2018 18:25
3,56_166917,56,166917,02-01-2018 08:15
4,56_172838,56,172838,02-01-2018 08:12


In [5]:
camp.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...
1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...
2,31,Conference,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,http://r.newsletters.analyticsvidhya.com/7usym...
3,32,Conference,24,19,7,1,\r\n \r\nHi ?\r\n \r\nBefore I dive into why y...,A.I. & Machine Learning: 5 reasons why you sho...,http://r.newsletters.analyticsvidhya.com/7uthl...
4,33,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...


In [6]:
train = pd.merge(train, camp, on='campaign_id')
test = pd.merge(test, camp, on='campaign_id')
print(train.shape, test.shape)

(1023191, 14) (773858, 12)


In [9]:
train.columns

Index(['id', 'user_id', 'campaign_id', 'send_date', 'is_open', 'is_click',
       'communication_type', 'total_links', 'no_of_internal_links',
       'no_of_images', 'no_of_sections', 'email_body', 'subject', 'email_url'],
      dtype='object')

In [10]:
test.columns

Index(['id', 'campaign_id', 'user_id', 'send_date', 'communication_type',
       'total_links', 'no_of_internal_links', 'no_of_images', 'no_of_sections',
       'email_body', 'subject', 'email_url'],
      dtype='object')

In [11]:
# Checking for the ratio of missing value
Total = train.isnull().sum().sort_values(ascending=False)
percent = ((train.isnull().sum()/train.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :percent, 'Missing Total':Total})
missing_data.head(10)

,Missing Ratio,Missing Total
email_url,0.0,0
subject,0.0,0
email_body,0.0,0
no_of_sections,0.0,0
no_of_images,0.0,0
no_of_internal_links,0.0,0
total_links,0.0,0
communication_type,0.0,0
is_click,0.0,0
is_open,0.0,0


In [12]:
# Checking for the ratio of missing value
Total = test.isnull().sum().sort_values(ascending=False)
percent = ((test.isnull().sum()/test.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :percent, 'Missing Total':Total})
missing_data.head(10)

,Missing Ratio,Missing Total
email_url,0.0,0
subject,0.0,0
email_body,0.0,0
no_of_sections,0.0,0
no_of_images,0.0,0
no_of_internal_links,0.0,0
total_links,0.0,0
communication_type,0.0,0
send_date,0.0,0
user_id,0.0,0


In [13]:
# Process date variable
# 01-02-2018 22:35
# 2017-08-21 00:00:01
train['send_date'] = pd.to_datetime(train['send_date'])
test['send_date'] = pd.to_datetime(test['send_date'])

In [15]:
# Feature Enginerring to Create New variables for Day of Week and Day part
#train
train['Weekday'] = (train['send_date'].dt.dayofweek).astype(int)
train['Dayhour'] = (train.send_date.apply(lambda x: x.hour)).astype(int)

#test
test['Weekday'] = (test['send_date'].dt.dayofweek).astype(int)
test['Dayhour'] = (test.send_date.apply(lambda x: x.hour)).astype(int)

In [18]:
# To remove Ordinality in both Weekday and DayHour variables convert into Categories
#train
train['Weekday'] = train['Weekday'].map( {0: 'Sun', 1: 'Mon', 2: 'Tue', 3: 'Wed',
                                              4: 'Thr', 5: 'Fri', 6: 'Sat'} ).astype(str)
train['Dayhour'] = train['Dayhour'].map( {0: 'AM1', 1: 'AM1', 2: 'AM1', 3: 'AM2', 4: 'AM2', 5: 'AM2', 
                                              6: 'AM3', 7: 'AM3', 8: 'AM3', 9: 'AM4', 10: 'AM4', 11: 'AM4', 12: 'PM1', 
                                              13: 'PM1', 14: 'PM1', 15: 'PM2', 16: 'PM2', 17: 'PM2', 18: 'PM3', 
                                              19: 'PM3', 20: 'PM3', 21: 'PM4', 22: 'PM4', 23: 'PM4'} ).astype(str)

#test
test['Weekday'] = test['Weekday'].map( {0: 'Sun', 1: 'Mon', 2: 'Tue', 3: 'Wed',
                                              4: 'Thr', 5: 'Fri', 6: 'Sat'} ).astype(str)
test['Dayhour'] = test['Dayhour'].map( {0: 'AM1', 1: 'AM1', 2: 'AM1', 3: 'AM2', 4: 'AM2', 5: 'AM2', 
                                              6: 'AM3', 7: 'AM3', 8: 'AM3', 9: 'AM4', 10: 'AM4', 11: 'AM4', 12: 'PM1', 
                                              13: 'PM1', 14: 'PM1', 15: 'PM2', 16: 'PM2', 17: 'PM2', 18: 'PM3', 
                                              19: 'PM3', 20: 'PM3', 21: 'PM4', 22: 'PM4', 23: 'PM4'} ).astype(str)

In [19]:
# Create final day time variable
train['DayPeriod'] = (train['Weekday'] + train['Dayhour']).astype(str)
test['DayPeriod'] = (test['Weekday'] + train['Dayhour']).astype(str)
del train['send_date'], train['Weekday'],train['Dayhour'],test['send_date'], test['Weekday'],test['Dayhour']

In [21]:
print (train.campaign_id.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.communication_type.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.user_id.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.DayPeriod.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.is_open.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.is_click.value_counts(normalize=True).head())

54    0.091814
53    0.083495
52    0.080298
49    0.079514
42    0.079411
Name: campaign_id, dtype: float64
------------------------------------------------------
Newsletter         0.440830
Conference         0.325094
Upcoming Events    0.145588
Others             0.049782
Hackathon          0.032511
Name: communication_type, dtype: float64
------------------------------------------------------
183177    0.000020
145022    0.000020
4118      0.000019
216700    0.000019
114180    0.000019
Name: user_id, dtype: float64
------------------------------------------------------
SatPM4    0.154950
FriPM1    0.119108
WedPM3    0.097015
WedPM2    0.082237
SunPM3    0.079411
Name: DayPeriod, dtype: float64
------------------------------------------------------
0    0.89954
1    0.10046
Name: is_open, dtype: float64
------------------------------------------------------
0    0.987508
1    0.012492
Name: is_click, dtype: float64


In [23]:
test.columns

Index(['id', 'campaign_id', 'user_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url', 'DayPeriod'],
      dtype='object')

In [26]:
# prep data for Base Model
target_train = train['is_click']
id_test = test['id']

del train['id'],train['user_id'],train['is_open'],train['is_click'],train['email_body'],train['subject'],train['email_url']
del test['id'],test['user_id'],test['email_body'],test['subject'],test['email_url']

print(train.shape,test.shape)

(1023191, 7) (773858, 7)


In [32]:
train.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,DayPeriod
0,42,Newsletter,88,79,13,4,SunPM3
1,42,Newsletter,88,79,13,4,SunPM3
2,42,Newsletter,88,79,13,4,SunPM3
3,42,Newsletter,88,79,13,4,SunPM3
4,42,Newsletter,88,79,13,4,SunPM3


In [34]:
train_cats = pd.get_dummies(data=train, columns=['campaign_id','communication_type','DayPeriod'], drop_first=True)
test_cats = pd.get_dummies(data=test, columns=['campaign_id','communication_type','DayPeriod'], drop_first=True)
print (train_cats.shape,test_cats.shape)

(1023191, 54) (773858, 54)


In [36]:
train_cats.head()

,total_links,no_of_internal_links,no_of_images,no_of_sections,campaign_id_30,campaign_id_31,campaign_id_32,campaign_id_33,campaign_id_34,campaign_id_35,...,DayPeriod_ThrAM4,DayPeriod_ThrPM1,DayPeriod_ThrPM2,DayPeriod_TueAM4,DayPeriod_TuePM1,DayPeriod_TuePM2,DayPeriod_WedPM1,DayPeriod_WedPM2,DayPeriod_WedPM3,DayPeriod_WedPM4
0,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
traindf = np.array(train_cats)
testdf = np.array(test_cats)

xgb_preds = []

In [38]:
K = 5
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [39]:
start = time()

for train_index, test_index in kf.split(traindf):
    train_X, valid_X = traindf[train_index], traindf[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
                  'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(testdf)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, 
                      maximize=True, verbose_eval=50, early_stopping_rounds=50)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

end = time()
print ('Time taken is:', end-start)

[0]	train-auc:0.542637	valid-auc:0.544322
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.56303	valid-auc:0.568659
Stopping. Best iteration:
[19]	train-auc:0.56303	valid-auc:0.568659

[0]	train-auc:0.535969	valid-auc:0.534908
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.556677	valid-auc:0.555072
Stopping. Best iteration:
[12]	train-auc:0.556677	valid-auc:0.555073

[0]	train-auc:0.527834	valid-auc:0.524043
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.557203	valid-auc:0.55294
[100]	train-auc:0.560252	valid-auc:0.553502
[150]	train-auc:0.575844	valid-auc:0.574871
[200]	train-auc:0.587702	valid-auc:0.5841
Stopping. Best iteration:
[160]	train-auc:0.58672

In [40]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

out = pd.DataFrame({'id': id_test, 'is_click': preds})
out.to_csv("pred1.csv", index=False)